# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#!jupyter nbextension enable --py gmaps
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("../output_data/weather_queries.csv")


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Create a list containing coordinates
coordinates = list(zip(df["Lat"],df["Lon"]))
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [4]:
# Assign the marker layer to a variable
heatmap = gmaps.heatmap_layer(coordinates, weights=df["Humidity"])
# Add the layer to the map
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
conditions = (df["Max Temp"]>70) & (df["Max Temp"]<80) & (df["Cloudiness"]==0)
narrowed_city_df = df[conditions]
narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
68,São Francisco,0,BR,1584495901,93,-15.95,-44.86,70.56,0.72
72,Bubaque,0,GW,1584495903,74,11.28,-15.83,77.79,18.72
244,Ormara,0,PK,1584495966,54,25.21,64.64,75.42,8.19
269,Ilhabela,0,BR,1584495967,82,-23.78,-45.36,77.23,4.32
270,Puerto Baquerizo Moreno,0,EC,1584495934,77,-0.90,-89.60,79.32,4.32
294,Manaure,0,CO,1584495981,84,11.78,-72.44,77.31,19.82
310,Port Macquarie,0,AU,1584495854,60,-31.43,152.92,77.00,2.24
312,Iracemápolis,0,BR,1584495702,83,-22.58,-47.52,71.60,13.87
358,Jining,0,CN,1584495999,27,35.41,116.58,70.57,11.34
411,Santiago del Estero,0,AR,1584496024,83,-27.80,-64.26,72.72,5.64


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:

hotel_name=[]
lat=[]
lon=[]

for (city_lat, city_lon) in zip(narrowed_city_df["Lat"],narrowed_city_df["Lon"]):


    # geocoordinates
    target_coordinates = f"{city_lat},{city_lon}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
  #      "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    places_data = response.json()

#    print(response.url)
    try:
        lat.append(places_data["results"][0]["geometry"]["location"]["lat"])

        lon.append(places_data["results"][0]["geometry"]["location"]["lng"])

        hotel_name.append(places_data["results"][0]["name"])
    except:
        print("No results, passing..")
        pass


hotel_df = pd.DataFrame({
    "Hotel Name":hotel_name,
    "Lat":lat,
    "Lon":lon
})



No results, passing..
No results, passing..
No results, passing..


In [7]:

hotel_df



,Hotel Name,Lat,Lon
0,HOTEL GREEN FISH ®,-15.953509,-44.847374
1,Ponta Anchaca,11.303644,-15.805113
2,Zero Point Ormara,25.249423,64.596317
3,Ilha Flat Hotel,-23.810683,-45.365574
4,hotel Casa Opuntia,-0.900209,-89.609922
5,Del mar vendra,11.780080,-72.438018
6,Sails Port Macquarie by Rydges,-31.427097,152.898781
7,Wanda Realm Jining,35.408238,116.604295
8,Hotel Casino Carlos V,-27.787853,-64.258545
9,Hôtel Le Méridien,18.210170,-73.755423


In [8]:
# Add marker layer ontop of heat map
hotel_coordinates = list(zip(hotel_df["Lat"],hotel_df["Lon"]))
hotel_coordinates
          

[(-15.9535094, -44.8473736),
 (11.303644, -15.805113),
 (25.2494232, 64.5963174),
 (-23.810683, -45.365574),
 (-0.9002089000000001, -89.6099224),
 (11.7800798, -72.43801839999999),
 (-31.4270974, 152.8987813),
 (35.4082377, 116.6042948),
 (-27.787853, -64.258545),
 (18.2101698, -73.7554226),
 (-20.4572926, -53.7577373),
 (25.0871074, 61.7705451),
 (24.8862337, 90.73584509999999)]

In [9]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(hotel_coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…